In [1]:
import sys
import os
import pandas as pd
from datetime import datetime, timedelta

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule_weather import generate_activity_list, Scheduler
from weather_constraints import get_daylight_windows, get_tide_windows

from plot_weather import plot_weather

In [2]:
#read in activity data from excel
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows"
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Activities", skiprows=1)
constraints_df = pd.read_excel(path, sheet_name="Constraints", skiprows=1)

activities = generate_activity_list(act_df, constraints_df)

c:\Users\webbb\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
target_activity = "Punch out of pilot"
input_completion_date = datetime(2025, 12, 9, 3, 0)

In [12]:
#generate daylight and tidal windows for start and end date

#windows - process for target date +/- 7 days
window_data_start = input_completion_date - timedelta(days=7)
window_data_start_end = input_completion_date + timedelta(days=7)

daylight_windows = get_daylight_windows(start_date=window_data_start, end_date=window_data_start_end, save_csv=None)

tidal_data_path = os.path.join(main_input_folder_path, "Reference/harwich_tide.csv")
hw_windows, lw_windows = get_tide_windows(tidal_data_path, start_date=window_data_start, end_date=window_data_start_end, 
                                          skiprows=2, datetime_col="Date/Time", height_col="Height(m)", 
                                          slack_window_after=1.5, slack_window_before=1.5, plot=False,
                                          save_csv=None)

c:\Users\webbb\Documents\python_local\WeatherOpsPlanner\src\weather_constraints.py:50: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



Loaded data from 2025-10-01 00:00:00 to 2026-10-31 23:50:00 with 57024 records.
Filtered data from 2025-12-02 03:00:00 to 2025-12-16 03:00:00 with 2017 records.


In [ ]:

# 1. Set your project start date
project_start = datetime(2025, 12, 9, 3, 0)  # Example: 1st Dec 2025, 07:00

# 2. Create the Scheduler
scheduler = Scheduler(
    activities=activities,
    weather_data=None,  # Add your weather DataFrame if available
    daylight_csv="sizewell_daylight_windows_december_2025.csv",   # Add daylight windows if available
    tide_csv="lowestoft_slack_water_windows_december_2025.csv",    # Add tide windows if available
    start_datetime=project_start
)

# 3. Run the schedule
scheduled_activities = scheduler.schedule(run_critical_path=True)

# 4. Export to DataFrame for review or export
schedule_df = scheduler.to_dataframe()

# Calculate the total duration by subtracting the earliest start from the latest end
total_duration = schedule_df['End'].max() - schedule_df['Start'].min()

# Print the total duration
print(f"Total duration of the schedule: {total_duration}")

schedule_df.head()


Total duration of the schedule: 12 days 08:07:30


,ID,Description,Duration (hours),Start,End,Group,Predecessor IDs,Constraints,Critical,Tide Window Mismatch
0,PM01,Mobilisation of Leask marine spread,5.0,2025-12-09 03:00:00,2025-12-09 08:00:00,Pipe Management,[],"{'daylight_required': False, 'tide_window_requ...",False,False
1,CTV01,CTV transfer of Personnel to JUB,2.0,2025-12-09 07:50:58,2025-12-09 09:50:58,Crew Transfer Vessel,[],"{'daylight_required': True, 'tide_window_requi...",False,False
2,HDD01,Punch out of pilot,3.0,2025-12-09 09:50:58,2025-12-09 12:50:58,Horizontal Directional Drilling,"[PM01, CTV01]","{'daylight_required': False, 'tide_window_requ...",False,False
3,CTV02,Bathymetric survey of pilot to confirm bore lo...,3.0,2025-12-10 07:52:04,2025-12-10 10:52:04,Crew Transfer Vessel,[HDD01],"{'daylight_required': True, 'tide_window_requi...",False,False
4,HDD02,Pull back of pilot,6.0,2025-12-09 12:50:58,2025-12-09 18:50:58,Horizontal Directional Drilling,[HDD01],"{'daylight_required': False, 'tide_window_requ...",False,False


In [6]:
# Load the tidal data
file_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows\scripts\Tide Data_Lowestoft_10min.CSV"
tide_df = pd.read_csv(file_path, skiprows=3, names=["DateTime", "Height"])

# Convert the 'DateTime' column to datetime format
tide_df['DateTime'] = pd.to_datetime(tide_df['DateTime'], errors='coerce')

# Filter the data to include only rows where the month is December
december_tide_df = tide_df[tide_df['DateTime'].dt.month == 12]


In [7]:
tide_window_df = pd.read_csv("lowestoft_slack_water_windows_december_2025.csv")

In [8]:
fig = plot_weather(schedule_df, scheduler.daylight_windows, december_tide_df, tide_window_df)

In [9]:
outputpath=os.path.join(main_input_folder_path, "pipe_pull_schedule.html")
fig.write_html(outputpath, include_plotlyjs='cdn')